In [1]:
wd <- dirname(dirname(dirname(dirname(getwd()))))
source(paste0(wd,"/mission_control/treasure_map.R"))

library(readxl)
library(dplyr)

I_DIR <- paste0(E_DIR, "/val_hebron/somatic/dna_datav2/")
O_DIR <- paste0(E_DIR, "/val_hebron/somatic/clean/")

Warning message:
“package ‘dplyr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### 0 - Read TMB files

In [2]:
setwd(I_DIR)
files <- list.files()

### 1 - Extract TMBs

In [3]:
tmbs <- data.frame()
for( i in  list.files()){
  my_data <- read_excel(i, sheet = 1)
  snps <- my_data$dbSNP
  ll <- length(snps)
  tmp <- data.frame( 
    patient = as.character(i), 
    check = snps[ll-1], 
    tmb = as.numeric(as.character(snps[ll])),
    vus = sum(my_data$Classification == "VUS",na.rm = TRUE),
    tot = sum(!is.na(my_data$Classification),na.rm = TRUE),
    rows = ll
  )    
  tmbs <- rbind( tmbs, tmp)
}

Warning message:
“Unknown or uninitialised column: `Classification`.”Warning message:
“Unknown or uninitialised column: `Classification`.”

#### Fix Lab ID

In [4]:
tmbs$patient <- as.character(tmbs$patient)
tmbs$LAB.ID <- unlist(lapply( tmbs$patient, function(i) paste(strsplit(i,".xlsx")[[1]][c(1)], collapse = "")))

#### Impute TMBs

In [5]:
tmbs$tmb <- as.numeric(as.character(tmbs$tmb))

### Output files

In [6]:
tmb_out <-
tmbs %>% transmute( 
    LAB.ID,
    vus, 
    tot,
    rows,
    tmb_raw = tmb,
    tmb = log(tmb+1)
)

### 2 - Send it!

In [7]:
saveRDS( tmb_out, paste0(O_DIR, "tmb_go.Rds"))